In [1]:
import numpy as np
import scipy
import os
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
np.random.seed(0)

In [2]:
# Read Files
def load_data(dir_path):
    '''
        Function to Read Data from mat Files
    '''
    
    mat_files = []
    for m in os.listdir(dir_path):
        if m.endswith('.mat'):
            mat_files.append(m)
    
    data = {}
    for sbj, file in enumerate(mat_files):
        file_path = os.path.join(dir_path, file)
        mat_data = scipy.io.loadmat(file_path)
        data[sbj] = (mat_data)

    return data

In [3]:
def FilterBandPass(CNT, Wn, Fs):    
    Wn = np.array([Wn[0] / (Fs/2), Wn[1] / (Fs/2)])
    
    # Filter Design
    order = 3
    type = 'bandpass'
    b, a = scipy.signal.butter(order, Wn, btype = 'bandpass')
    
    # Apply Filter
    CNT = scipy.signal.filtfilt(b, a, CNT, axis = 0)
    
    return CNT

In [4]:
# CAR Filter for Spatial Filtering
def CARFilter(Data):
    M = np.mean(Data, axis = 1)
    
    for i in range(Data.shape[1]):
        Data[:, i] = Data[:, i] - M
    
    return Data

In [5]:
def CSP(Train1, Train2, m):
    Rh = 0
    for i in range(Train1.shape[2]):
        Data = Train1[:, :, i]

        # Normalize Data
        M = np.mean(Data, axis = 0)

        for i in range(Data.shape[0]):
            Data[i, :] = Data[i, :] - M

        # Cov Matrix Calculation
        Rh += Data.T @ Data / np.trace(Data.T @ Data)

    Rf = 0
    for i in range(Train2.shape[2]):
        Data = Train2[:, :, i]

        # Normalize Data
        M = np.mean(Data, axis = 0)

        for i in range(Data.shape[0]):
            Data[i, :] = Data[i, :] - M

        # Cov Matrix Calculation
        Rf += Data.T @ Data / np.trace(Data.T @ Data)


    # Normalize Cov Matrix
    Rf /= Train2.shape[2]
    Rh /= Train1.shape[2]

    # Eigen Value Decomposition
    u, v = scipy.linalg.eig(Rh, Rf)
    # u = np.real(u)

    # Sort eigenvalues and eigenvectors by eigenvalues
    sorted_indices = np.argsort(u)[::-1]  # Sort in descending order
    u = u[sorted_indices]
    v = v[:, sorted_indices]

    Wm = list(range(m)) + list(range(v.shape[1] - m, v.shape[1]))
    W = v[:, Wm]

    return W

In [6]:
def RCSP(Train1, Train2, m, Alpha):
    Rh = 0
    for i in range(Train1.shape[2]):
        Data = Train1[:, :, i]

        # Normalize Data
        M = np.mean(Data, axis = 0)

        for i in range(Data.shape[0]):
            Data[i, :] = Data[i, :] - M

        # Cov Matrix Calculation
        Rh += Data.T @ Data / np.trace(Data.T @ Data)

    Rf = 0
    for i in range(Train2.shape[2]):
        Data = Train2[:, :, i]

        # Normalize Data
        M = np.mean(Data, axis = 0)

        for i in range(Data.shape[0]):
            Data[i, :] = Data[i, :] - M

        # Cov Matrix Calculation
        Rf += Data.T @ Data / np.trace(Data.T @ Data)


    # Normalize Cov Matrix
    Rf /= Train2.shape[2]
    Rh /= Train1.shape[2]

    R1 = Rh + Alpha * np.identity(Rh.shape[0])
    R2 = Rf + Alpha * np.identity(Rh.shape[0])
    
    # Eigen Value Decomposition
    u1, v1 = scipy.linalg.eig(Rh, R2)
    # u = np.real(u)
    
    # Sort eigenvalues and eigenvectors by eigenvalues
    sorted_indices = np.argsort(u1)[::-1]  # Sort in descending order
    u1 = u1[sorted_indices]
    v1 = v1[:, sorted_indices]
    
    
    # Eigen Value Decomposition
    u2, v2 = scipy.linalg.eig(Rf, R1)
    # u = np.real(u)
    
    # Sort eigenvalues and eigenvectors by eigenvalues
    sorted_indices = np.argsort(u2)[::-1]  # Sort in descending order
    u2 = u2[sorted_indices]
    v2 = v2[:, sorted_indices]
    
    W = np.concatenate((v1[:, :m], v2[:, :m]), axis = 1)

    return W

In [7]:
dir_path = './Dataset'
data = load_data(dir_path)
np.random.seed(0)

AvailbaleDataKeys = data.keys()
# AvailbaleDataKeys = [6]

for key in list(AvailbaleDataKeys):
    
    # Read Data
    cnt = data[key]['cnt']
    nfo = data[key]['nfo']
    mrk = data[key]['mrk']
    fs = nfo['fs'][0][0][0][0]
    
    ## Filtering Data
    cnt = FilterBandPass(cnt, [8, 30], fs)
    
    # Spatial Filtering
    cnt = CARFilter(cnt)
    
    # Trial Seperation
    Ltr = 4 * fs
    Pos = mrk['pos'][0][0][0]
    Group = mrk['y'][0][0][0]
    
    data1 = np.zeros((Ltr, cnt.shape[1], len(Group)//2))
    data2 = np.zeros((Ltr, cnt.shape[1], len(Group)//2))
    c1 = 0
    c2 = 0
    
    for i in range(len(Pos)):
        Idx = range(Pos[i], Pos[i] + Ltr)
        Trial = cnt[Idx, :]
        
        if Group[i] == 1:
            data1[:, :, c1] = Trial
            c1 += 1
        elif Group[i] == -1:
            data2[:, :, c2] = Trial
            c2 += 1

    ## Train and Test Data Separation
    # Index Creation
    TestDiv = 0.2
    Idx = np.random.choice(data1.shape[2], size=int(TestDiv * data1.shape[2]), replace=False)
    TestIdx = np.zeros(data1.shape[2], dtype=bool)
    TestIdx[Idx] = True
    
    TrainIdx = ~TestIdx
    
    # Data Seperation
    Train1 = data1[:, :, TrainIdx]
    Train2 = data2[:, :, TrainIdx]
    
    Test1 = data1[:, :, TestIdx]
    Test2 = data2[:, :, TestIdx]
    
    # CSP Weight Matrix Creation
    M = 2
    AlphaList = [10e-10, 10e-9, 10e-8, 10e-7, 10e-6, 10e-5, 10e-4, 10e-3, 10e-2, 10e-1]
    
    Accuracy = []
    ConfMat = []
    for Alpha in AlphaList:

        W = RCSP(Train1, Train2, M, Alpha)


        # Train Feature Extraction
        Train = np.concatenate((Train1, Train2), axis = 2)    
        FeatureTrain = np.zeros((2 * M, Train.shape[2]))

        for i in range(Train.shape[2]):
            tmp = Train[:, :, i].T
            tmp = W.T @ tmp
            FeatureTrain[:, i] = np.var(tmp, axis = 1)

        # Test Feature Extraction
        Test = np.concatenate((Test1, Test2), axis = 2) 
        FeatureTest = np.zeros((2 * M, Test.shape[2]))

        for i in range(Test.shape[2]):
            tmp = Test[:, :, i].T
            tmp = W.T @ tmp
            FeatureTest[:, i] = np.var(tmp, axis = 1)

        # Label Creation
        LabelTrain = np.concatenate((np.ones(Train1.shape[2]), 2* np.ones(Train2.shape[2])))
        LabelTest = np.concatenate((np.ones(Test1.shape[2]), 2* np.ones(Test2.shape[2])))


        # Train and Test Classifier
        mdl = KNeighborsClassifier(n_neighbors=5)
        # mdl = svm.SVC(kernel='linear', C = 1.0)

        mdl.fit(FeatureTrain.T, LabelTrain)
        LabelPredict = mdl.predict(FeatureTest.T)

        Acc = accuracy_score(LabelTest, LabelPredict)
        ConfMat.append(confusion_matrix(LabelTest, LabelPredict))
        
        Accuracy.append(Acc)
    
    Accuracy = np.array(Accuracy)
    Alpha = AlphaList[np.argmax(Accuracy)]
    ConfusionMatrix = ConfMat[np.argmax(Accuracy)]

    # Print Metrics
    print(f'Subject {key + 1}')
    print("Accuracy : ", np.max(Accuracy), f' With Alpha of {Alpha}')
    print(f"Class 1 Accuracy : {(np.diag(ConfusionMatrix) / ConfusionMatrix.sum(axis=1))[0] * 100}")
    print(f"Class 2 Accuracy : {(np.diag(ConfusionMatrix) / ConfusionMatrix.sum(axis=1))[1] * 100}")
    print("Confusion Matrix : \n", ConfusionMatrix)
    print('-----------------------------------------------------------------')

Subject 1
Accuracy :  0.9  With Alpha of 1e-09
Class 1 Accuracy : 90.0
Class 2 Accuracy : 90.0
Confusion Matrix : 
 [[18  2]
 [ 2 18]]
-----------------------------------------------------------------
Subject 2
Accuracy :  0.675  With Alpha of 0.0001
Class 1 Accuracy : 70.0
Class 2 Accuracy : 65.0
Confusion Matrix : 
 [[14  6]
 [ 7 13]]
-----------------------------------------------------------------
Subject 3
Accuracy :  0.7  With Alpha of 1e-09
Class 1 Accuracy : 70.0
Class 2 Accuracy : 70.0
Confusion Matrix : 
 [[14  6]
 [ 6 14]]
-----------------------------------------------------------------
Subject 4
Accuracy :  0.8  With Alpha of 1e-05
Class 1 Accuracy : 75.0
Class 2 Accuracy : 85.0
Confusion Matrix : 
 [[15  5]
 [ 3 17]]
-----------------------------------------------------------------
Subject 5
Accuracy :  0.925  With Alpha of 0.001
Class 1 Accuracy : 95.0
Class 2 Accuracy : 90.0
Confusion Matrix : 
 [[19  1]
 [ 2 18]]
--------------------------------------------------------